# Interfaces: From Protocols to ABCs
Python was already highly successful before ABCs were introduced, and most existing code does not use them at all. <br>
One of the most fundamental interfaces in Python is the sequence protocol. The interpreter goes out of its way to handle objects that provide even a minimal implementation of that protocol, as the next section demonstrates.


## Python Digs Sequences


In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]


In [2]:
f = Foo()

In [3]:
for i in f: print(i)

0
10
20


In [4]:
20 in f

True

In [5]:
15 in f

False

There is no method __iter__ yet Foo instances are iterable because - as a fallback - when Python sees a __getitem__ method, it tries to iterate over the object by calling that method with integer indexes starting with 0. Because Python is smart enough to iterate over Foo instances, it can also make the in operator work even if Foo has no __contains__ method: it does a full scan to check if an item is present. In summary, given the importance of the sequence protocol, in the absence __iter__
and __contains__ Python still manages to make iteration and the in operator work by
invoking __getitem__.

## Monkey-Patching to Implement a Protocol at Runtime